In [4]:
import os
import glob
import warnings
warnings.filterwarnings("ignore")

## for data
import pandas as pd  #(0.25.1)
import numpy as np #(1.16.4)
## for plotting
import matplotlib.pyplot as plt  #(3.1.2)
import seaborn as sns  #(0.9.0)
## for preprocessing
import re
import nltk  #(3.4.5)
import contractions  #(0.0.18)
## for textrank
import gensim  #(3.8.1)
## for evaluation
#import rouge  #(1.0.0)
import difflib
## for seq2seq
from tensorflow.keras import callbacks, models, layers, preprocessing as kprocessing #(2.6.0)
import tensorflow as tf
## for bart
import transformers  #(3.0.1)

2023-03-22 17:55:02.176450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 17:55:20.336866: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.2/lib64:/shared/centos7/nodejs/14.15.4/lib
2023-03-22 17:55:20.337597: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.2/lib64:/shared/centos7/nodejs/

In [28]:
print("Physical Devices : {}".format(tf.config.list_physical_devices()))

SyntaxError: unexpected EOF while parsing (155111618.py, line 1)

In [26]:
dataset_path = "dataset"

In [27]:
categories = os.listdir(dataset_path)
#categories

In [18]:
print("Reading the data")
wiki_data = { "Title": [] , "Article" : [] , "Category" : []  , "Summary" : []}
for category in categories:
    article_path = dataset_path+"/"+category+"/Article/"
    summary_path = dataset_path+"/"+category+"/Summary/"

    for file in glob.glob(article_path+"/*.txt"):
        file_name = os.path.basename(file)
        title = file_name.replace(".txt","")
        
        with open(file,'r') as f:        
            wiki_data["Article"].append(f.read())
            wiki_data["Category"].append(category)
            
        with open(summary_path+file_name,'r') as f:        
            wiki_data["Summary"].append(f.read())
            wiki_data["Title"].append(title)

In [19]:
print("Creating the dataframe")
wikiDF = pd.DataFrame( data = wiki_data )
wikiDF.reset_index(drop=True, inplace=True)

In [20]:
wikiDF.head(5)

,Title,Article,Category,Summary
0,Masaccio,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De...",People,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De..."
1,Hokusai,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ...",People,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ..."
2,Steven Spielberg,"Steven Allan Spielberg (; born December 18, 1...",People,"Steven Allan Spielberg (; born December 18, 1..."
3,Che Guevara,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]...",People,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]..."
4,Jöns Jacob Berzelius,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...,People,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...


In [21]:
print("Saving the data in CSV")
wikiDF.to_csv("Wikipedia-Level-4-Articles.csv")

In [22]:
## create stopwords
lst_stopwords = nltk.corpus.stopwords.words("english")
## add words that are too frequent
lst_stopwords = lst_stopwords + ["cnn","say","said","new"]

## cleaning function
def utils_preprocess_text(txt):
    punkt=True
    lower=True
    slang=True
    lst_stopword=True
    stemm=True
    lemm=True
    ### separate sentences with '. '
    txt = re.sub(r'\.(?=[^ \W\d])', '. ', str(txt))
    ### remove punctuations and characters
    txt = re.sub(r'[^\w\s]', '', txt) if punkt is True else txt
    ### strip
    txt = " ".join([word.strip() for word in txt.split()])
    ### lowercase
    txt = txt.lower() if lower is True else txt
    ### slang
    txt = contractions.fix(txt) if slang is True else txt   
    ### tokenize (convert from string to list)
    lst_txt = txt.split()
    ### stemming (remove -ing, -ly, ...)
    if stemm is True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_txt = [ps.stem(word) for word in lst_txt]
    ### lemmatization (convert the word into root word)
    if lemm is True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_txt = [lem.lemmatize(word) for word in lst_txt]
    ### remove Stopwords
    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in 
                   lst_stopwords]
    ### back to string
    txt = " ".join(lst_txt)
    return txt


In [23]:
text = wikiDF.iloc[0]["Article"]

In [24]:
type(text)

str

In [16]:
print("Cleaning the data")
wikiDF["CleanedText"] = wikiDF["Article"].apply(utils_preprocess_text)

KeyboardInterrupt: 

In [ ]:
wikiDF.to_csv("Wikipedia-Level-4-Articles-Cleaned.csv")